Objective: Use mujoco sim data to map the boundary of pose space 

Methodology: 
* go through each timestep of each trial of data 
* if non-zero contact and contact is below top surface (non-inclusive), then save pose to list 
* plot the high dimensional manifold as a 2D manifold in 3D space by fixing 3 dimensions 

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import glob 
import os 
import pickle 
from scipy.spatial.transform import Rotation as R 

In [3]:
# go through each timestep of each trial of data 

peg_geom = "plug_3_pin" # "cross", "plug_3_pin"  

dir_results = "/media/rp/Elements/abhay_ws/mujoco_contact_graph_generation/results/plug_3_pin_data_v2" 
# dir_results = "/media/rp/Elements/abhay_ws/mujoco_contact_graph_generation/results/cross_peg_data_v2" 
dir_pkl = dir_results + "/pkl" 
pkl_files = sorted(glob.glob(os.path.join(dir_pkl, "*.pkl")), key=os.path.getmtime)

dir_save = dir_pkl.removesuffix("pkl") + "processed_data"
if not os.path.exists(dir_save): 
    os.makedirs(dir_save)

# list of all contact state history 
N_timesteps = 500 
N_trials = 10_000 
N_trials = len(pkl_files) if len(pkl_files) < N_trials else N_trials
pkl_files = pkl_files[:N_trials] 
pose_boundary_list = [] 

for i, pkl_file in enumerate(pkl_files): 

    # Read the pickle file
    with open(pkl_file, 'rb') as f:
        data = pickle.load(f)

    # unpack data 
    state_hist = data['state_hist'] 
    contact_pos = data['contact_pos'] 

    # check if there is contact within the hole area, if so, add the pose to the list 
    for j, contact_pos_j in enumerate(contact_pos): # iterate through each time step 
        if len(contact_pos_j) > 0: # if there is contact 
            for k, contact_pos_hole_frame in enumerate(contact_pos_j): # iterate through each contact at current time step 
                if peg_geom == "cross":
                    if contact_pos_hole_frame[2] < 0 and max(contact_pos_hole_frame[:2]) < 0.012: # if contact is below the surface and within hole area 
                        peg_pose = state_hist[j, 1:8] 
                        pose_boundary_list.append(peg_pose) 
                        continue # don't need to check pose again 
                elif peg_geom == "plug_3_pin": 
                    x_contact = contact_pos_hole_frame[0] * 1E3
                    y_contact = contact_pos_hole_frame[1] * 1E3
                    z_contact = contact_pos_hole_frame[2] * 1E3 
                    if z_contact < 0 and (((x_contact-0)**2 + (y_contact - -10)**2 < 8) or ((x_contact - -6)**2 + (y_contact - 3.5)**2 < 10) or ((x_contact - 6)**2 + (y_contact - 3.5)**2 < 8)): # if contact is below the surface and within hole area 
                        peg_pose = state_hist[j, 1:8] 
                        pose_boundary_list.append(peg_pose) 
                        continue # don't need to check pose again 
    # print progress rate every 10% of total iterations 
    if (i+1) % np.floor(len(pkl_files)/10) == 0: 
        print(f"Completion Progress: {i+1}/{len(pkl_files)}")  

# convert list to dataframe 
pose_boundary_df = pd.DataFrame(pose_boundary_list, columns=['x', 'y', 'z', 'qw', 'qx', 'qy', 'qz']) 

# convert quaternion to euler angles 
quaternions = pose_boundary_df[['qw', 'qx', 'qy', 'qz']].values 
euler_angles = R.from_quat(quaternions, scalar_first=True).as_euler("xyz", degrees=True) 
pose_boundary_df['a'] = euler_angles[:,2]
pose_boundary_df['b'] = euler_angles[:,1]
pose_boundary_df['c'] = euler_angles[:,0] 

# convert position from meters to millimeters 
pose_boundary_df[['x', 'y', 'z']] *= 1000 

# save the dataframe 
if not os.path.exists(dir_save): 
    os.makedirs(dir_save)
pose_boundary_df.to_csv(os.path.join(dir_save, "pose_boundary_data_10k.csv"), index=False)  

Completion Progress: 1000/10000
Completion Progress: 2000/10000
Completion Progress: 3000/10000
Completion Progress: 4000/10000
Completion Progress: 5000/10000
Completion Progress: 6000/10000
Completion Progress: 7000/10000
Completion Progress: 8000/10000
Completion Progress: 9000/10000
Completion Progress: 10000/10000
